# Unit 5: Programming with Spark SQL and DataFrames

## Contents
```
5.1 Structured data processing
5.2 SQL
5.3 DataFrames
5.4 Performance improvement
5.5 SparkSession
5.6 Creating DataFrames
5.7 Saving a DataFrame
5.8 DataFrame operations
5.9 Query Strings
5.10 Column Expressions
5.11 DataFrames and RDDs
5.12 SQL Queries
```

## Structured data processing
Spark SQL is a Spark module for **structured data processing**.

Unlike the basic Spark RDD API, the interfaces provided by Spark SQL provide Spark with more information about the structure of both the data and the computation being performed. Internally, Spark SQL uses this extra information to **perform additional optimizations**.

There are several ways to interact with Spark SQL including **SQL, the DataFrames API and the Datasets API**. 

**When computing a result the same execution engine is used, independently of which API/language you are using to express the computation.** This unification means that developers can easily switch back and forth between the various APIs based on which provides the most natural way to express a given transformation.

## SQL
One use of Spark SQL is to **execute SQL queries written using either ANSI SQL syntax or HiveQL**. Spark SQL can also be used to read data from an existing Hive installation. For more on how to configure this feature, please refer to the Hive Tables section. When running SQL from within another programming language the results will be returned as a DataFrame. You can also interact with the SQL interface using the command-line or over JDBC/ODBC.

Reference: [Spark SQL, DataFrames and Datasets Guide](https://spark.apache.org/docs/2.4.0/sql-programming-guide.html)

## DataFrames
A DataFrame is a distributed collection of data **organized into named columns**. It is conceptually equivalent to a table in a relational database or a data frame in R/Python, but with richer optimizations under the hood. DataFrames can be constructed from a wide array of sources such as: structured data files, tables in Hive, external databases, or existing RDDs.

## Performance improvement
Spark SQL and DataFrames take advantadge of the fact that they are using structured data to optimize the performance using the [Catalyst query optimizer](https://databricks.com/blog/2015/04/13/deep-dive-into-spark-sqls-catalyst-optimizer.html).

![Performance comparison](https://databricks.com/wp-content/uploads/2015/02/Screen-Shot-2015-02-16-at-9.46.39-AM.png)
Reference: [Performance improvements in Spark](https://databricks.com/blog/2015/04/24/recent-performance-improvements-in-apache-spark-sql-python-dataframes-and-more.html)


## SparkSession
In Spark 2 to use Spark SQL or DataFrames we use a **SparkSession** object as the main entry point to the API, in a similar way as we use the SparkContext (sc) as the main entry point to the RDD API. In fact, inside the SparkSession object we have an attribute that points to the SparkContext.

Just be aware, that when using previous versions of Spark, to use Spark SQL or DataFrames instead of a **SparkSession** you have to use a **SQLContext** object, and that there are two implementations of the SQLContext object:
* SQLContext: basic
* HiveContext: more advanced
  * It is able to read Hive tables directly
  * Supports HiveQL language

Spark 2 unifies the SQLContext and HiveContext directly in the SparkSession object simplifying its usage.

In our case the Spark Shell provides us automatically with a SparkSession object called **spark**.

In [1]:
type(spark)

pyspark.sql.session.SparkSession

## Creating DataFrames

### Creating a DataFrame from an existing file in JSON format

Reading an existing file in **JSON** format:

In [2]:
processes = spark.read.json('datasets/pacct_20170701_c66.json')

In [3]:
processes.printSchema()

root
 |-- command: string (nullable = true)
 |-- etime: double (nullable = true)
 |-- exitcode: long (nullable = true)
 |-- flag: long (nullable = true)
 |-- gid: long (nullable = true)
 |-- host: string (nullable = true)
 |-- io: long (nullable = true)
 |-- majflt: long (nullable = true)
 |-- mem: long (nullable = true)
 |-- minflt: long (nullable = true)
 |-- pid: long (nullable = true)
 |-- ppid: long (nullable = true)
 |-- rw: long (nullable = true)
 |-- stime: double (nullable = true)
 |-- swaps: long (nullable = true)
 |-- tm_hour: long (nullable = true)
 |-- tm_isdst: long (nullable = true)
 |-- tm_mday: long (nullable = true)
 |-- tm_min: long (nullable = true)
 |-- tm_mon: long (nullable = true)
 |-- tm_sec: long (nullable = true)
 |-- tm_wday: long (nullable = true)
 |-- tm_yday: long (nullable = true)
 |-- tm_year: long (nullable = true)
 |-- tty: long (nullable = true)
 |-- uid: long (nullable = true)
 |-- utime: double (nullable = true)
 |-- version: long (nullable = true)

### Creating a DataFrame from an existing file in Parquet format

Reading an existing file in **parquet** format:

In [4]:
processes2 = spark.read.parquet('datasets/pacct_20170701.parquet')

In [5]:
processes2.printSchema()

root
 |-- host: string (nullable = true)
 |-- flag: integer (nullable = true)
 |-- version: integer (nullable = true)
 |-- tty: integer (nullable = true)
 |-- exitcode: integer (nullable = true)
 |-- uid: integer (nullable = true)
 |-- gid: integer (nullable = true)
 |-- pid: integer (nullable = true)
 |-- ppid: integer (nullable = true)
 |-- tm_year: integer (nullable = true)
 |-- tm_mon: integer (nullable = true)
 |-- tm_mday: integer (nullable = true)
 |-- tm_hour: integer (nullable = true)
 |-- tm_min: integer (nullable = true)
 |-- tm_sec: integer (nullable = true)
 |-- tm_wday: integer (nullable = true)
 |-- tm_yday: integer (nullable = true)
 |-- tm_isdst: integer (nullable = true)
 |-- etime: decimal(10,2) (nullable = true)
 |-- utime: decimal(10,2) (nullable = true)
 |-- stime: decimal(10,2) (nullable = true)
 |-- mem: integer (nullable = true)
 |-- io: integer (nullable = true)
 |-- rw: integer (nullable = true)
 |-- minflt: integer (nullable = true)
 |-- majflt: integer (nul

### Creating a DataFrame from an existing file in CSV format

Reading an existing file in **CSV** format:

```
[jlopez@cdh61-login8 NYC_taxi_trip_records]$ head yellow_tripdata_2018-12.csv 
VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount

1,2018-12-01 00:28:22,2018-12-01 00:44:07,2,2.50,1,N,148,234,1,12,0.5,0.5,3.95,0,0.3,17.25
1,2018-12-01 00:52:29,2018-12-01 01:11:37,3,2.30,1,N,170,144,1,13,0.5,0.5,2.85,0,0.3,17.15
2,2018-12-01 00:12:52,2018-12-01 00:36:23,1,.00,1,N,113,193,2,2.5,0.5,0.5,0,0,0.3,3.8
1,2018-12-01 00:35:08,2018-12-01 00:43:11,1,3.90,1,N,95,92,1,12.5,0.5,0.5,2.75,0,0.3,16.55
```

In [6]:
trips = spark.read.csv('datasets/NYC_taxi_trip_records')

In [7]:
trips.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)



In order to use the appropriate column names we must indicate that the first line of the file contains a header with column names:

In [8]:
trips = spark.read.csv('datasets/NYC_taxi_trip_records', header=True)

In [9]:
trips.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)



And finally, in order to infer correctly the dates as timestamps instead of strings we must specify the datetime format used: 

In [10]:
trips = spark.read.csv('datasets/NYC_taxi_trip_records', header=True,
                       inferSchema=True, timestampFormat="yyyy-MM-dd HH:mm:ss")

In [11]:
trips.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



### Creating a DataFrame from an existing file in other formats

Other formats like Avro, HBase, etc. are also supported, in some cases using third party data sources.

### Creating a DataFrame from a Hive table

In [12]:
trips = spark.sql('select * from demos.trips limit 1000')

In [13]:
trips.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



In [14]:
trips.count()

1000

### Creating a DataFrame from an RDD

A DataFrame is built from **an RDD that has a collection of Row objects** using the toDF() function.

In [15]:
from pyspark.sql import Row

In [16]:
peopleRDD = sc.parallelize([('Aroa', 18, 'student'), ('Lara', 15, 'student'), ('Susana', 35, 'teacher')])

In [17]:
type(peopleRDD)

pyspark.rdd.RDD

In this case we have to convert the collection of tuples in a collection of Rows and then we can transform it in an DF.

In [18]:
peopleDF = peopleRDD.map(lambda p: Row(name=p[0], age=int(p[1]), profession=p[2])).toDF()

In [19]:
type(peopleDF)

pyspark.sql.dataframe.DataFrame

In [20]:
peopleDF.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)
 |-- profession: string (nullable = true)



## Saving a DataFrame

In [21]:
trips.write.parquet('trips_parquet')

It creates a directory in HDFS and stores there the data using one file per partition using parquet format:
```bash
[jlopez@cdh61-login8 ~]$ hdfs dfs -ls trips_parquet
Found 2 items
-rw-r--r--   3 jlopez cesga          0 2019-06-18 13:02 trips_parquet/_SUCCESS
-rw-r--r--   3 jlopez cesga      31791 2019-06-18 13:02 trips_parquet/part-00000-08bf4845-3cbe-4713-a658-30de22647a94-c000.snappy.parquet
```

We can also use other output formats like JSON, CSV or Avro:

In [22]:
trips.write.json('trips_json')

```bash
[jlopez@cdh61-login8 ~]$ hdfs dfs -ls trips_json
Found 2 items
-rw-r--r--   3 jlopez cesga          0 2019-06-18 13:02 trips_json/_SUCCESS
-rw-r--r--   3 jlopez cesga     390452 2019-06-18 13:02 trips_json/part-00000-39341409-85fb-4e11-8e08-334a8d28dd17-c000.json
```

In [23]:
trips.write.csv('trips_csv')

```bash
[jlopez@cdh61-login8 ~]$ hdfs dfs -ls trips_csv
Found 2 items
-rw-r--r--   3 jlopez cesga          0 2019-06-18 13:02 trips_csv/_SUCCESS
-rw-r--r--   3 jlopez cesga     111995 2019-06-18 13:02 trips_csv/part-00000-aa464f26-3659-4b59-81b6-326f000513c1-c000.csv
```

In [24]:
trips.write.format('avro').save('trips_avro')

```bash
[jlopez@cdh61-login8 ~]$ hdfs dfs -ls trips_avro
Found 2 items
-rw-r--r--   3 jlopez cesga          0 2019-06-18 13:02 trips_avro/_SUCCESS
-rw-r--r--   3 jlopez cesga      39188 2019-06-18 13:02 trips_avro/part-00000-e4b4f013-5608-4de2-bee4-1f9a951a20d2-c000.avro
```

We can even export the dataframe back to Hive as a Hive table, for example to export it to a table in the `jlopez` database:

In [25]:
trips.write.saveAsTable('jlopez.trips')

AnalysisException: u'Table `jlopez`.`trips` already exists.;'

Then we can query that table directly from Hive.

## DataFrame Operations

As in the case of RDDs where we had transformations and actions in this case we have:
* Queries: **lazy** transformations that create a new DataFrame
* Actions: trigger the execution of queries and return the data to the driver

### Actions

#### show

Displays the first n rows

In [ ]:
peopleDF.show(2)

#### take

Returns the first n rows

In [ ]:
peopleDF.take(2)

#### collect

Returns all rows

In [ ]:
peopleDF.collect()

#### count

In [ ]:
peopleDF.count()

### Queries

#### distinct

In [ ]:
df = spark.createDataFrame([Row(name='aroa', age=17), Row(name='aroa', age=17), Row(name='lara', age=14)])

In [ ]:
df.show()

In [ ]:
df.distinct().show()

#### limit

In [ ]:
processes.count()

In [ ]:
processes_chunk = processes.limit(100)

In [ ]:
processes_chunk.count()

#### where/filter

The **where** and **filter** operations are equivalent: where() is an alias for filter().

In [ ]:
peopleDF.where('name = "Aroa"').show()

The query is expressed using a Query String (see Query Strings section below).

#### select

We can do projections, for example reducing the number of columns:

In [ ]:
processes.select('pid', 'command', 'etime')

We can also do transformations (see Column Expressions section below):

In [ ]:
peopleDF.select(peopleDF.name, peopleDF.age + 100).show()

#### orderBy

In [ ]:
peopleDF.orderBy(peopleDF.age.desc()).show()

The order is controlled by a Column Expression: .asc() and .desc() are column expressions (see the Column Expressions section below).

#### groupBy

We can group data (it returns a GroupedData object with additional operations):

In [ ]:
peopleDF.groupBy('profession')

And then we can perform operations on grouped data:

* Calculate the maximum/minimum:

In [ ]:
peopleDF.groupBy('profession').max('age').show()

* Calculate the mean:

In [ ]:
peopleDF.groupBy('profession').mean('age').show()

* Calculate the sum:

In [ ]:
peopleDF.groupBy('profession').sum('age').show()

Reference: [Available GroupedData operations](https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html#pyspark.sql.GroupedData)

#### join

We can join DataFrames in a similar way as we did with PairRDDs:

In [ ]:
professionsDF = spark.createDataFrame([Row(name='student', description='A person engaged in study'), Row(name='teacher', description='A person whose occupation is teaching')])

In [ ]:
peopleDF.join(professionsDF, peopleDF.profession == professionsDF.name).show()

### Query Strings

It is important to understand what type of **Query Strings** we can use:

In [ ]:
peopleDF.where(peopleDF.age > 10).show()

In [ ]:
peopleDF.where('age > 10').show()

In [ ]:
peopleDF.where(peopleDF['age'] > 10).show()

### Column Expressions

Some queries like select, sort, join or where can take column expressions.

In [ ]:
# A column
peopleDF.name

We can operate on columns:

In [ ]:
peopleDF.select(peopleDF.name, peopleDF.age + 100).show()

In [ ]:
peopleDF.orderBy(peopleDF.age.desc()).show()

In [ ]:
peopleDF.orderBy(peopleDF.age.asc()).show()

In [ ]:
peopleDF.orderBy(peopleDF.profession.like('stu%')).show()

You can set multiple conditions using operators: & | ~

## DataFrames and RDDs

Sometimes it is useful to use a DataFrame as an RDD so you all the flexibility of the RDD API.

It is very easy to access the underlying RDD of Rows, it is exposed under the **.rdd** property:

In [ ]:
import math
peopleDF.rdd.map(lambda row: (row.name, math.sqrt(row.age))).collect()

## SQL Queries

Spark 2.4 supports both ANSI-SQL as well as Hive QL queries.

For example the SELECT statement supports the following semantics:

```
SELECT [ALL|DISTINCT] named_expression[, named_expression, ...]
    FROM relation[, relation, ...]
    [lateral_view[, lateral_view, ...]]
    [WHERE boolean_expression]
    [aggregation [HAVING boolean_expression]]
    [ORDER BY sort_expressions]
    [CLUSTER BY expressions]
    [DISTRIBUTE BY expressions]
    [SORT BY sort_expressions]
    [WINDOW named_window[, WINDOW named_window, ...]]
    [LIMIT num_rows]

named_expression:
    : expression [AS alias]

relation:
    | join_relation
    | (table_name|query|relation) [sample] [AS alias]
    : VALUES (expressions)[, (expressions), ...]
          [AS (column_name[, column_name, ...])]

expressions:
    : expression[, expression, ...]

sort_expressions:
    : expression [ASC|DESC][, expression [ASC|DESC], ...]
```

To express conditions `case...when...then...end` staments are also supported.

The first thing we have to do before launching SQL queries is to **create a local temporary view of the DataFrame**:

In [ ]:
peopleDF.createOrReplaceTempView('people')

After doing this we can start launching SQL queries against the DataFrame data just using the view name we have created:

In [ ]:
spark.sql('SELECT * FROM people WHERE age > 20').show()

In [ ]:
spark.sql('''SELECT * FROM people WHERE name LIKE "Ar%"''').show()

And this would be an example of how to use conditions using `case...when...then...end` statements to create a new column:

In [ ]:
spark.sql('''SELECT name, age,
  CASE WHEN age < 18 THEN 'No'
       WHEN age >= 18 THEN 'Yes'
  END AS adult
FROM people''').show()

Spark SQL also provides a bunch of functions that you can use:

```
SHOW FUNCTIONS
```
These functions can be used inside SQL statements or using the DataFrames API and you can even create your own functions. More details in the next two sections.

## List of built-in functions

There are several builtin functions that can be useful to operate on Columns:

* [List of built-in functions](http://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html#module-pyspark.sql.functions)

When we are in doubt about how to do some transformation it is useful to check this list before proceeding to use the underlying RDD directly. For example we have methods for:
* abs
* avg
* cos
* concat
* regexp_extract
* regexp_replace
* sum
* when
* otherwise
* lit

## UDFs

When there is no builtin function available, we can create our own user-defined functions (UDF) that will allow us to use custom python code to operate on Columns.

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
is_adult = udf(lambda n: 1 if n > 18 else 0, IntegerType())

In [ ]:
peopleDF.select(peopleDF.name, is_adult(peopleDF.age).alias('adult')).collect()

## Exercises
Review the documentation:
* [pyspark.sql documentation](https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html)

Exercises:
* Unit 5 Working with meteorological data, now using DataFrames. You can also try to solve it using SQL.
* Unit 5 Sentiment Analysis: Review the Sentiment Analysis notebook that makes use of DataFrames and Spark ML